<a id='top'></a>
# World Food Programme: food price and affordability analysis
### Data scraping

This script illustrates the first phase of the 'World Food Programme: Food Price and Affordability Analysis' project. The project is using data from the World Food Programme Price Database.  

The World Food Programme is monitoring the price of staple foods included in a [basic food basket](https://humanitarianglobal.com/key-tools-and-types-of-information-required-for-monitoring-the-adequacy-of-ration-in-emergencies/#:~:text=Food%20Basket%20Monitoring%20(FBM)&text=A%20systematic%20sample%20of%20households,each%20of%20their%20food%20items) across 99 countries. In the World Food Programme Price Database, the data is stored and can be downloaded at a country level. 

I aim to merge the available data to gain a global perspective on food price dynamics. The following Python code was used to create a combined food price data set in 3 steps: 

1. [Scraping](#scrape)

With the list of URLs from ['wfp_countries_global.csv'](https://www.kaggle.com/datasets/jocelyndumlao/global-food-prices?select=wfp_countries_global.csv) I can gain access to each page on [OCHA Services](https://data.humdata.org/dataset?dataseries_name=WFP+-+Food+Prices) where the food price data for each country can be downloaded in a.csv format. Using BeautifulSoup, I've scraped each of the pages to retrieve the links to the individual country data.

2. [Using a for loop to download 99 individual data sets](#download)

Using the list of links to the individual data sets, I've created a for loop to download all the files. 

3. [Combining the data into a single data set](#concat)  

Using pd.concat, I combined the data from the 99 individual data sets into one single dataframe and exported it for further analysis.


In [45]:
import pandas as pd
import numpy as np
import os
from time import sleep
from bs4 import BeautifulSoup
import requests
from os.path import basename
import urllib.request

<a id='scrape'></a>
## Scraping
[Back to top](#top)

In [2]:
path=r'C:\Users\frauz\Documents\Python Projects\Final Project\Data' #creating a path the store the files

df_countries=pd.read_csv(os.path.join(path,'wfp_countries_global.csv'), index_col=False) #import the dataset with all the countries and links

In [6]:
df_countries.head()

,countryiso3,url,start_date,end_date
0,#country+code,#country+url,#date+start,#date+end
1,AFG,https://data.humdata.org/dataset/wfp-food-pric...,2000-01-15 00:00:00+00:00,2024-02-15 23:59:59+00:00
2,AGO,https://data.humdata.org/dataset/wfp-food-pric...,2008-01-15 00:00:00+00:00,2021-05-15 23:59:59+00:00
3,ARG,https://data.humdata.org/dataset/wfp-food-pric...,2005-01-15 00:00:00+00:00,2022-12-15 23:59:59+00:00
4,ARM,https://data.humdata.org/dataset/wfp-food-pric...,1996-01-15 00:00:00+00:00,2024-01-15 23:59:59+00:00


In [11]:
df_countries=df_countries.drop([0]) # dropping the unnecesarry row

df_countries.head()

In [35]:
url_list=df_countries['url'].to_list() # creating a list of links
len(url_list)

99

In [54]:
country_list=df_countries['countryiso3'].to_list() # creating a list of countries
country_list

['AFG',
 'AGO',
 'ARG',
 'ARM',
 'AZE',
 'BDI',
 'BEN',
 'BFA',
 'BGD',
 'BLR',
 'BOL',
 'BTN',
 'CAF',
 'CHN',
 'CIV',
 'CMR',
 'COD',
 'COG',
 'COL',
 'CPV',
 'CRI',
 'DJI',
 'DOM',
 'DZA',
 'ECU',
 'EGY',
 'ERI',
 'ETH',
 'FJI',
 'GAB',
 'GEO',
 'GHA',
 'GIN',
 'GMB',
 'GNB',
 'GTM',
 'HND',
 'HTI',
 'IDN',
 'IND',
 'IRN',
 'IRQ',
 'JOR',
 'JPN',
 'KAZ',
 'KEN',
 'KGZ',
 'KHM',
 'LAO',
 'LBN',
 'LBR',
 'LBY',
 'LKA',
 'LSO',
 'MDA',
 'MDG',
 'MEX',
 'MLI',
 'MMR',
 'MNG',
 'MOZ',
 'MRT',
 'MWI',
 'NAM',
 'NER',
 'NGA',
 'NIC',
 'NPL',
 'PAK',
 'PAN',
 'PER',
 'PHL',
 'PRY',
 'PSE',
 'RUS',
 'RWA',
 'SDN',
 'SEN',
 'SLE',
 'SLV',
 'SOM',
 'SSD',
 'SWZ',
 'SYR',
 'TCD',
 'TGO',
 'THA',
 'TJK',
 'TLS',
 'TUR',
 'TZA',
 'UGA',
 'UKR',
 'VEN',
 'VNM',
 'YEM',
 'ZAF',
 'ZMB',
 'ZWE']

In [86]:
url_list[1] # extract 1 link we are going to use for testing the code 

'https://data.humdata.org/dataset/wfp-food-prices-for-angola'

In [23]:
page=requests.get('https://data.humdata.org/dataset/wfp-food-prices-for-angola').text # open the link
print(page) # get everything from the page

<!DOCTYPE html>
<!--[if IE 7]> <html lang="en_AU" class="ie ie7"> <![endif]-->
<!--[if IE 8]> <html lang="en_AU" class="ie ie8"> <![endif]-->
<!--[if IE 9]> <html lang="en_AU" class="ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en_AU"> <!--<![endif]-->
  <head>

    <script type="application/javascript">
      /**
       * A map holding all the information to be sent to the analytics servers
       * @type {object}
       */
      var analyticsInfo = {
          'organizationName': 'wfp',
          'organizationId': '3ecac442-7fed-448d-8f78-b385ef6f84e7',
          'groupNames': ["ago"],
          'groupIds': ["ago"],
          'isCod': false,
          'isIndicator': false,
          'isArchived': false,
          'datasetName': 'wfp-food-prices-for-angola',
          'datasetId': 'f6155f17-bded-468c-b926-b9de5034d2ad',
          'pageTitle': 'Angola - Food Prices',
          'datasetAvailability': 'public',
          'authenticated': 'false'

      };
    </script>

    
   

In [34]:
soup=BeautifulSoup(page) # turn page content into soup
for link in soup.find_all('a'): # loop through all links
    csv_link=str(link.get('href')) # turn links into strings
    if csv_link.endswith('csv') and ('wfp_food_prices' in csv_link) and ('qc' not in csv_link): #select links that meet the conditions
        print(csv_link)

/dataset/f6155f17-bded-468c-b926-b9de5034d2ad/resource/8d87dc41-26b9-4928-86ec-75f95480e18d/download/wfp_food_prices_ago.csv


In [39]:
# Now I can create a loop to collect all 99 csv links I need to download the data

final_list=[]   # create a list to collect all the links

for i in url_list: # code has to loop through each of 99 links on the url_list
    page_content=requests.get(i).text # turns the content of the page into text
    soup=BeautifulSoup(page_content) # turns content into soup
    for link in soup.find_all('a'): # now it needs to loop through every link on the page
        csv_link=str(link.get('href')) # if the link meets the criteria: ends with .csv, contains 'wfp_food_prices' and doesn't contain 'qc'
        if csv_link.endswith('csv') and ('wfp_food_prices' in csv_link) and ('qc' not in csv_link): 
            final_list.append(csv_link) #link gets appended to the list
    print(final_list)

['/dataset/a246cbac-42d5-47b2-ba75-ac66f69e83de/resource/03e6ce5d-03a2-4e60-8d04-afa39c5972f4/download/wfp_food_prices_afg.csv', '/dataset/f6155f17-bded-468c-b926-b9de5034d2ad/resource/8d87dc41-26b9-4928-86ec-75f95480e18d/download/wfp_food_prices_ago.csv', '/dataset/49dc23aa-dd97-4985-adaf-b79f1c6ec0f4/resource/ea099434-3274-4314-800b-41458d1e9746/download/wfp_food_prices_arg.csv', '/dataset/605f01e2-32a3-4d94-a718-0c79c4c613ff/resource/0798b658-a35d-42c7-bc37-84b28e6d8410/download/wfp_food_prices_arm.csv', '/dataset/79dc8689-3d66-4f4e-b95a-64dd7acd763f/resource/e78439c2-5e3f-4988-b751-bb758f1c6dc3/download/wfp_food_prices_aze.csv', '/dataset/e7e5df38-adb6-4843-a053-24e119b27afc/resource/efa227dd-2fd5-437e-b79d-130af02b769d/download/wfp_food_prices_bdi.csv', '/dataset/66c7d54e-0c3b-45e5-9a46-07ea6f195093/resource/7da1ea0a-56c7-450a-af2c-d477745fc856/download/wfp_food_prices_ben.csv', '/dataset/bfd82e1f-0296-48a8-ac28-c11e028be5ed/resource/0eca67d6-e297-4f5e-9132-7dc42891b749/download/w

In [58]:
# saving the list in case I might need it later

file=open('csv_links.txt','w') # creating a new file to write down all links
for item in final_list:
    file.write('https://data.humdata.org/'+item+'\n') #write each link in the text file
file.close()

<a id='download'></a>
## Downloading the files
[Back to top](#top)

In [99]:
# I want to figure out how to extract country iso code from the link so I can use it as the name of the file while downloading

url='/dataset/f6155f17-bded-468c-b926-b9de5034d2ad/resource/8d87dc41-26b9-4928-86ec-75f95480e18d/download/wfp_food_prices_ago.csv'
name=url.rsplit('_',1)
name[-1].strip()

'ago.csv'

In [100]:
# Now I want to use the links to download the data. First, I try with one link

base_link='https://data.humdata.org/'
url='/dataset/f6155f17-bded-468c-b926-b9de5034d2ad/resource/8d87dc41-26b9-4928-86ec-75f95480e18d/download/wfp_food_prices_ago.csv'
url_list=url.rsplit('_',1)
filename=url_list[-1].strip()
urllib.request.urlretrieve(base_link+url,os.path.join(path,name))

('C:\\Users\\frauz\\Documents\\Python Projects\\Final Project\\Data\\ago.csv',
 <http.client.HTTPMessage at 0x197d5a14c10>)

In [57]:
# before I try to loop through 99 links, I want to tra with just 3
test_list=final_list[:3] # creating a sub-list to test the loop first
test_list

['/dataset/a246cbac-42d5-47b2-ba75-ac66f69e83de/resource/03e6ce5d-03a2-4e60-8d04-afa39c5972f4/download/wfp_food_prices_afg.csv',
 '/dataset/f6155f17-bded-468c-b926-b9de5034d2ad/resource/8d87dc41-26b9-4928-86ec-75f95480e18d/download/wfp_food_prices_ago.csv',
 '/dataset/49dc23aa-dd97-4985-adaf-b79f1c6ec0f4/resource/ea099434-3274-4314-800b-41458d1e9746/download/wfp_food_prices_arg.csv']

In [104]:
# Now I try to download 3 files
base_link='https://data.humdata.org/'
for url in test_list:
    url_list=url.rsplit('_',1)
    name=url_list[-1].strip()
    file = open(name, "w")
    urllib.request.urlretrieve(base_link+url, os.path.join(path, name))
    file.close

In [105]:
# It's time to download the rest of the files

rest_list=final_list[3:]
len(rest_list)

96

In [106]:
 base_link='https://data.humdata.org/'
for url in rest_list:
    url_list=url.rsplit('_',1)
    name=url_list[-1].strip()
    file = open(name, "w")
    urllib.request.urlretrieve(base_link+url, os.path.join(path, name))
    file.close

<a id='concat'></a>
## Combining the files into a single data set
[Back to top](#top)

In [108]:
# let's import one data set and check its structure

df_afg=pd.read_csv(os.path.join(path, 'afg.csv'), index_col=False)

In [111]:
col_list=df_afg.columns.to_list()

In [159]:
# Now I want to combine the 99 individual data sets into one

data_global = pd.concat(pd.read_csv(os.path.join(path, f'{iso}.csv')).assign(iso=iso)
                     for iso in country_list)

C:\Users\frauz\AppData\Local\Temp\ipykernel_13240\1729988833.py:3: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_global = pd.concat(pd.read_csv(os.path.join(path, f'{iso}.csv')).assign(iso=iso)
C:\Users\frauz\AppData\Local\Temp\ipykernel_13240\1729988833.py:3: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_global = pd.concat(pd.read_csv(os.path.join(path, f'{iso}.csv')).assign(iso=iso)
C:\Users\frauz\AppData\Local\Temp\ipykernel_13240\1729988833.py:3: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_global = pd.concat(pd.read_csv(os.path.join(path, f'{iso}.csv')).assign(iso=iso)
C:\Users\frauz\AppData\Local\Temp\ipykernel_13240\1729988833.py:3: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data_global = pd.concat(pd.read_cs

In [160]:
data_global.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,iso
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,AFG
1,2000-01-15,Badakhshan,Faiz Abad,Fayzabad,37.116638,70.580022,non-food,"Wage (non-qualified labour, non-agricultural)",Day,actual,Retail,AFN,150000.0,3205.7378,AFG
2,2000-01-15,Balkh,Mazar-e-Sharif,Mazar,36.725116,67.109571,non-food,"Wage (non-qualified labour, non-agricultural)",Day,actual,Retail,AFN,100000.0,2137.1586,AFG
3,2000-01-15,Hirat,Hirat,Hirat,34.346944,62.198333,cereals and tubers,Bread,KG,actual,Retail,AFN,15.63,0.334,AFG
4,2000-01-15,Hirat,Hirat,Hirat,34.346944,62.198333,cereals and tubers,Wheat,KG,actual,Retail,AFN,9.13,0.1951,AFG


In [119]:
data_global.shape

(3542949, 15)

Saving the warning for future reference: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.

In [132]:
# I can see that there are records in the dataframe that contain the names of the columns. I need to remove them

data_global.loc[data_global['price']=='#value']

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,iso
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,AFG
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,AGO
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,ARG
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,ARM
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,AZE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,VNM
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,YEM
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,ZAF
0,#date,#adm1+name,#adm2+name,#loc+market+name,#geo+lat,#geo+lon,#item+type,#item+name,#item+unit,#item+price+flag,#item+price+type,#currency,#value,#value+usd,ZMB


In [135]:
# Since I haven't reset the index yet, all unnecesary rows are indexed 0

df_global=data_global.drop(data_global.index[0])

In [136]:
df_global.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,iso
1,2000-01-15,Badakhshan,Faiz Abad,Fayzabad,37.116638,70.580022,non-food,"Wage (non-qualified labour, non-agricultural)",Day,actual,Retail,AFN,150000.0,3205.7378,AFG
2,2000-01-15,Balkh,Mazar-e-Sharif,Mazar,36.725116,67.109571,non-food,"Wage (non-qualified labour, non-agricultural)",Day,actual,Retail,AFN,100000.0,2137.1586,AFG
3,2000-01-15,Hirat,Hirat,Hirat,34.346944,62.198333,cereals and tubers,Bread,KG,actual,Retail,AFN,15.63,0.334,AFG
4,2000-01-15,Hirat,Hirat,Hirat,34.346944,62.198333,cereals and tubers,Wheat,KG,actual,Retail,AFN,9.13,0.1951,AFG
5,2000-01-15,Hirat,Hirat,Hirat,34.346944,62.198333,cereals and tubers,Wheat flour,KG,actual,Retail,AFN,10.06,0.215,AFG


In [137]:
# Now I can reset the index of the dataframe

df_global.reset_index(inplace=True)

In [140]:
df_global.index

RangeIndex(start=0, stop=3542751, step=1)

In [143]:
# Changing data types
df_global=df_global.astype({'price':'float','usdprice':'float','latitude':'float','longitude':'float'})
print(df_global.dtypes)

index          int64
date          object
admin1        object
admin2        object
market        object
latitude     float64
longitude    float64
category      object
commodity     object
unit          object
priceflag     object
pricetype     object
currency      object
price        float64
usdprice     float64
iso           object
dtype: object


In [145]:
df_global.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,index,latitude,longitude,price,usdprice
count,3542751.00000,3515876.00000,3515876.00000,3542751.00000,3538809.00000
mean,49728.05970,13.59380,37.25309,6791.21714,13.54921
std,50759.71417,16.61378,46.51168,83441.09058,1075.81118
min,1.00000,-34.61000,-107.38600,0.00000,0.00000
25%,13118.00000,1.72680,8.68138,1.54000,0.16780
50%,33592.00000,12.32634,34.25617,130.50000,0.74070
75%,67029.00000,27.65266,71.55583,1200.00000,1.99500
max,250774.00000,59.93000,179.37736,17250000.00000,599999.99110


In [151]:
#The total count of rows in the final dataframe has first surprised me. I'm counting the number of rows per country

df_count_by_country=df_global['iso'].value_counts()
df_count_by_country

IDN    250774
SYR    240503
IND    177336
GMB    149692
PHL    136253
        ...  
CRI       330
AZE       149
VEN       109
ERI       100
GEO        80
Name: iso, Length: 99, dtype: int64

In [152]:
df_count_by_country.describe() 

count        99.000000
mean      35785.363636
std       47975.981543
min          80.000000
25%        2655.000000
50%       17092.000000
75%       51962.000000
max      250774.000000
Name: iso, dtype: float64

#### It looks like Indonesia is the country with the highest number of rows in the dataframe - 250774, while Georgia has only 80 records. To confirm that concatenation has worked correctly, I'll import the individual data sets and check their length.


In [153]:
df_idn=pd.read_csv(os.path.join(path,'idn.csv'))

C:\Users\frauz\AppData\Local\Temp\ipykernel_13240\3309933425.py:1: DtypeWarning: Columns (4,5,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_idn=pd.read_csv(os.path.join(path,'idn.csv'))


In [155]:
df_idn.shape

(250775, 14)

In [156]:
df_geo=pd.read_csv(os.path.join(path,'geo.csv'))

In [157]:
df_geo.shape

(81, 14)

#### It looks like both data sets have been concatenated accurately. At this point, I assume the same for the remaining 97 data sets.

In [158]:
# The completed dataframe can now be exported and used in the next project phase
df_global.to_pickle(os.path.join(path,'global_food_prices.pkl'))